# BIDS-Microscopy: empty ometiff generator to create an example dataset

The present notebook can generate an empty OME-TIFF image (only with metadata) from a sample png. The png has no metadata, so the OME-XML is created with a typed string. From this metadata, a BIDS compatible JSON file is generated. The JSON fields are filled from the OME-TIFF's metadata as a dictionary.

In [ ]:
import tifffile
from ctypes import *
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.11/bin/libgobject-2.0-0.dll")  # Change for the path where vips-dev-8.11 is
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.11/bin/libvips-42.dll")        # idem
import pyvips
import json

## Generating the empty OME-TIFF file

In [ ]:
empty_png = pyvips.Image.new_from_array([0])

empty_png = empty_png.copy()
empty_png.set_type(pyvips.GValue.gint_type, "page-height", empty_png.height)
empty_png.set_type(pyvips.GValue.gstr_type, "image-description",
f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Contributors
            Authors="Etienne Bergeron, Marie-Hélène Bourget"
            Acknowledgements="Some acknowledgements.">
    </Contributors>
    <Dataset
            SubjectScheme="Allen Mouse Brain Atlas Version2"
            License="MIT"
            NumberOfSubjects="number of lines in participants.tsv"
            NumberOfSamples="number of lines in samples.tsv"
            BIDSVersion="1.6.0 - BEP031 v0.0.5"
            ExperimentalLogFile="n/a">
            <Methods
                SampleStaining="n/a"
                Environment="exvivo"
                SamplePrimaryAntibody="n/a"
                SampleSecondaryAntibody="n/a"
                SampleEmbedding="n/a"
                SampleFixation="4% paraformaldehyde, 2% glutaraldehyde"
                SampleStainingDescription="n/a"
                SampleExtractionProtocol="n/a"
                SampleExtractionInstitution="n/a"
                SliceThickness="{1}"
                ShrinkageFactor="{2}"/>
            <TechnicalInfo
                ImageAcquisitionProtocol="n/a"
                SoftwareVersions="n/a"/>  
    </Dataset>
    <Funders
            Funding="n/a"
            FundingReferenceIdentifier="n/a">
    </Funders>
    <Image>
            <Pixels
                PhysicalSizeX="{1}"
                PhysicalSizeXUnit="um"
                PhysicalSizeY="{1}"
                PhysicalSizeYUnit="um"
                PhysicalSizeZ="{1}"             
                PhysicalSizeZUnit="um"
                DimensionOrder="XYCZT"
                SizeC="{empty_png.bands}"
                SizeT="{1}"
                SizeX="{empty_png.width}"
                SizeY="{empty_png.height}"
                SizeZ="{1}"
                Type="uint8">
            </Pixels>
    </Image>
    <Instrument
            MicroscopeType="SEM"
            ManufacturerAndModel="Hamamatsu C9600-12"
            ObjectiveImmersion="Oil"
            ObjectiveNA="n/a"
            ObjectiveMagnification="n/a"
            DetectorType="CCD"
            IlluminationType="Gas"
            IlluminationWavelength="n/a"
            DetectionWavelength="n/a"
            SampleTemperature="n/a">
    </Instrument>
    <Publications
            RelatedIdentifier="n/a"
            Citation="n/a">
    </Publications>
    <Specimen
            LocalID="'participant_id"
            NCBITaxonomy="mus musculus"
            Age="n/a"
            AgeUnit="years"
            AgeCategory="n/a"
            AgeMin="n/a"
            AgeMax="n/a"
            Sex="M"
            Handedness="n/a"
            Genotype="n/a"
            OrganLocalID="n/a"
            OrganName="n/a"
            SampleLocalID="n/a"
            Derived_from="n/a"
            Atlas="n/a"
            SampleType="n/a"
            Location="CSPINE"
            ExperimentalGroup="n/a"
            Strain="n/a"
            RRIDStrain="n/a">
    </Specimen>
</OME>""")

empty_png.tiffsave('sub-example_sample-1_sem.ome.tif',compression="jpeg", tile=True, tile_width=512, tile_height=512, pyramid=True)

## Generating the JSON file

In [ ]:
with tifffile.TiffFile('sub-example_sample-1_sem.ome.tif', is_ome=True) as ome:
    print(ome.ome_metadata)
    meta_dict = tifffile.xml2dict(ome.ome_metadata)['OME']
    print(meta_dict)

JSON_dict = {
        "Manufacturer" : meta_dict['Instrument']['ManufacturerAndModel'],
        "ManufacturersModelName" : meta_dict['Instrument']['ManufacturerAndModel'],
        "DeviceSerialNumber" : "n/a",
        "StationName" : "n/a",
        "SoftwareVersions" : meta_dict['Dataset']['TechnicalInfo']['SoftwareVersions'],
        "InstitutionName" : "n/a",
        "InstitutionAddress" : "n/a",
        "InstitutionalDepartmentName" : "n/a",
        "BodyPart" : meta_dict['Specimen']['Location'],
        "BodyPartDetails" : "n/a",
        "BodyPartDetailsOntology" : "n/a",
        "Environment" : meta_dict['Dataset']['Methods']['Environment'],
        "SampleEmbedding" : meta_dict['Dataset']['Methods']['SampleEmbedding'],
        "SampleFixation" : meta_dict['Dataset']['Methods']['SampleFixation'],
        "SampleStaining" : meta_dict['Dataset']['Methods']['SampleStaining'],
        "SamplePrimaryAntibody" : meta_dict['Dataset']['Methods']['SamplePrimaryAntibody'],
        "SampleSecondaryAntibody" : meta_dict['Dataset']['Methods']['SampleSecondaryAntibody'],
        "SliceThickness" : meta_dict['Dataset']['Methods']['SliceThickness'],
        "SampleExtractionProtocol" : meta_dict['Dataset']['Methods']['SampleExtractionProtocol'],
        "SampleExtractionInstitution" : meta_dict['Dataset']['Methods']['SampleExtractionInstitution'],
        "ShrinkageFactor" : meta_dict['Dataset']['Methods']['ShrinkageFactor'],
        "PixelSize" : [meta_dict['Image']['Pixels']['PhysicalSizeX'], meta_dict['Image']['Pixels']['PhysicalSizeY'], meta_dict['Image']['Pixels']['PhysicalSizeZ']],
        "PixelSizeUnits" : meta_dict['Image']['Pixels']['PhysicalSizeXUnit'],
        "ImageAcquisitionProtocol" : meta_dict['Dataset']['TechnicalInfo']['SoftwareVersions'],
        "OtherAcquisitionParameters" : "n/a"
}

with open('sub-example_sample-1_sem.json', 'w') as fp:
    json.dump(JSON_dict, fp, indent=8)